<a href="https://colab.research.google.com/github/Abhiy28/Self-Organization/blob/main/ANTS_TRAIL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from manim import *

In [ ]:
a = np.array([1, 0, 0])
def RandomVel():
    th = np.random.uniform(-3.14, 3.14)
    ROT3D = np.array([[np.cos(th), np.sin(th), 0], [-np.sin(th), np.cos(th), 0], [0, 0, 0]])
    vel = np.matmul(ROT3D, a)
    return vel


N = 10  # number of ants
'''We start with initially zero pheromones and the then each ant lays the pheromones'''
M = 100

sa = 0.05  # size of ants
EdgeU = 4
EdgeD = -4
EdgeL = -7

EdgeR = 7
speed = 1  # speed of ants
size = 0.2*sa  # size of the pheromone representation
T_e = 100  # meantime of evaporation

PoissonWandering = 0.02  # Exploratory part of ant motion. Ranges from 0 to 1
GaussVariance = 0.1
GaussMean = 0
Theta = np.pi / 4  # Defining the range of sight by R and Theta ... Detecting a pheromone particle

R = 10 * sa
lam_r = 0.01*speed  # Deposition rate
lam_t = 0.1 # Trail recruitment rate

DepositionRate = 0.002


def wondering(ANT):
    th = GaussVariance * np.random.randn() + GaussMean
    ROT3D = [[np.cos(th), np.sin(th), 0], [-np.sin(th), np.cos(th), 0], [0, 0, 0]]
    ANT.vel = np.matmul(ANT.vel, ROT3D)
    return ANT


def Detecting_Walls(ANT):
    # dth = np.random.uniform(-np.pi/4, np.pi/4)
    v = np.array(ANT.pos < np.array([EdgeL, EdgeD, 0]), dtype=int) - np.array(np.array([EdgeR, EdgeU, 0]) < ANT.pos,
                                                                                  dtype=float)
    ANT.pos = np.array([EdgeR*v[0], EdgeU*v[1], 0])+np.array([ANT.pos[0]*(1-v[0]**2), ANT.pos[1]*(1-v[1]**2), 0])
        #ROT3D = np.array([[np.cos(dth), np.sin(dth), 0], [-np.sin(dth), np.cos(dth), 0], [0, 0, 0]])
        #ANT.vel = np.matmul(-ANT.vel, ROT3D)

    return ANT


def Trial_recruitment(ANT, ph):
    neighbour = ANT.neighbor(ph)
    if any(neighbour):
        neighbour = neighbour / np.sum(neighbour)
        t = [-np.pi / 6, 0, np.pi / 6]
        th = np.random.choice(t, p=neighbour)
        M = np.array([[np.cos(th), np.sin(th), 0], [-np.sin(th), np.cos(th), 0], [0, 0, 0]])
        ANT.vel = np.matmul(M, ANT.vel)
        return ANT


def Pheromone_deposition(Mobject, ANT_object):
    v = np.random.uniform(0, 1, N)
    v = np.nonzero(v)[0]
    Mobject.add(*[Dot(ANT_object[i].pos, color=TEAL_A, radius=size) for i in v])
    return Mobject


def Update_pheromones(Mobject):
    s = len(Mobject)
    v = np.random.uniform(0, 1, s)
    t = np.array(v < 1 / T_e, dtype = int)
    t = np.nonzero(t)[0]
    rl = VGroup()
    rl.add(*[Mobject[i] for i in t])
    for r in rl:
      Mobject.remove(r)

    Pheromone_deposition(Mobject, ANTS_object)


'''Initialization of ants'''

#POS = np.array([[np.random.uniform(-1, 1), np.random.uniform(-1, 1), 0] for i in range(N)])
POS = np.array([[np.random.uniform(EdgeL, EdgeR), np.random.uniform(EdgeD, EdgeU), 0] for i in range(N)])  # Random initial position of ants
VEL = speed * np.array([RandomVel() for i in range(N)])  # Initial velocities of ants in random directions

'''Initialization of pheromones'''

PheromonePosition = np.array([[np.random.uniform(EdgeL, EdgeR), np.random.uniform(EdgeD, EdgeU), 0] for i in range(M)])

pheromones = VGroup()
pheromones.add_updater(Update_pheromones)
for i in range(M):
    pheromones.add(Dot(PheromonePosition[i], color=TEAL_A, radius=size))



'''___'''


class Ant:
    def __init__(self, pos, vel, i):
        self.rep = Dot(pos, color=BLACK, radius=sa)
        self.pos = pos
        self.vel = vel
        self.label = i

    def neighbor(self, ph):
        s = len(ph)
        n = np.array([np.linalg.norm(ph[i].get_center() - self.pos) < R for i in range(s)])
        n = np.nonzero(n)[0]
        m = 0
        l = 0
        r = 0
        for i in n:
          phpos = ph[i].get_center()
          phi = np.arcsin(
          np.linalg.norm(np.cross(phpos - self.pos, self.vel)) /(speed*np.linalg.norm(phpos - self.pos)))
          if np.pi / 2 - Theta / 3 < phi < np.pi / 2 + Theta / 3:
            m += 1
          if np.pi / 2 - Theta < phi < np.pi / 2 - Theta / 3 :
            l += 1
          if np.pi / 2 + Theta / 3 < phi < np.pi / 2 + Theta:
            r += 1

        return np.array([l, m, r])

    def motion(self, Mobject, dt):
        self.pos += self.vel * dt

        if any(self.pos < np.array([EdgeL, EdgeD, 0])) or any(np.array([EdgeR, EdgeU, 0]) < self.pos):
            Detecting_Walls(self)

        a = np.random.uniform()
        if a < PoissonWandering:
            wondering(self)

        b = np.random.uniform()
        if b < lam_t:
            Trial_recruitment(self, pheromones)
        Mobject.move_to(self.pos)


ANTS_object = [Ant(POS[i], VEL[i], i) for i in range(N)]

ANTS = VGroup()
for i in range(N):
    ANTS.add(ANTS_object[i].rep.add_updater(ANTS_object[i].motion))


'''____'''




'____'

In [ ]:
%%manim -qm -v WARNING AntTrail
class AntTrail(Scene):
    def construct(self):
        self.camera.background_color = DARK_BROWN
        self.add(pheromones)
        self.add(ANTS)
        self.wait(5)
